In [121]:
import sys, os
import numpy as np
sys.path.append(os.pardir)

In [122]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [123]:
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    
    return y

cross_entropy_error (loss function)

In [124]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y + 1e-7)) / batch_size

numerical gradient function to calculate the gradients of function f by x

In [129]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    if x.ndim == 1: #in case x is (n,).
        x = x.reshape(1, x.size)
        grad = grad.reshape(1, x.size)
    
    for i in range(len(x)):
        for j in range(len(x[0])):
            tmp_val = x[i][j]
            x[i][j] = tmp_val + h
            fxh1 = f(x)
    
            x[i][j] = tmp_val - h
            fxh2 = f(x)
    
            grad[i][j] = (fxh1 - fxh2) / (2*h)
            x[i][j] = tmp_val
    
    return grad

In [130]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        
        self.params={}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2) + b2
        z2 = sigmoid(a2)
        y = softmax(z2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def numerical_gradient(self, x, t): #calculate the gradient value for W1, b1, W2, b2 respectively.
        def loss_W(W):
            return self.loss(x,t) #loss_W = lambda W: self.loss(x,t) instead.
        
        grads={}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
            

In [ ]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)
x = np.random.randn(100, 784)
y = net.predict(x)
t = np.random.randn(100, 10)

grads = net.numerical_gradient(x, t) #takes too much time to finish the calculation